In [2]:
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, GRU, Embedding, Dropout, Activation, BatchNormalization, AveragePooling1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from Attention import *
import h5py

/disk1/home/sum5275/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/disk1/home/sum5275/anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
data = pd.read_csv('../data/train.csv')

In [4]:
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,22256635,"Nonsense? kiss off, geek. what I said is true...",1,0,0,0,0,0
1,27450690,"""\n\n Please do not vandalize pages, as you di...",0,0,0,0,0,0
2,54037174,"""\n\n """"Points of interest"""" \n\nI removed the...",0,0,0,0,0,0
3,77493077,Asking some his nationality is a Racial offenc...,0,0,0,0,0,0
4,79357270,The reader here is not going by my say so for ...,0,0,0,0,0,0


In [5]:
classes = data.columns[2:].values

In [6]:
train_comments = data['comment_text'].fillna('UNK')
labels = data.loc[:, classes].values

In [7]:
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH = 100

In [8]:
tokenizer = Tokenizer(num_words = MAX_NB_WORDS)
tokenizer.fit_on_texts(train_comments)

In [9]:
train_sequences = tokenizer.texts_to_sequences(train_comments)
X_train = pad_sequences(train_sequences, maxlen = MAX_SEQUENCE_LENGTH)

In [10]:
y_train = data[classes].values

In [11]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 153188 unique tokens.


In [12]:
! ls ../embeddings

glove.840B.300d.txt  glove.840B.300d.zip


In [13]:
embeddings_index = {}
f = open('../embeddings/glove.840B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    try:
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    except:
        print("error reading word", word)
f.close()

print('Found %s word vectors.' % len(embeddings_index))

error reading word .
error reading word at
error reading word .
error reading word to
error reading word .
error reading word .
error reading word email
error reading word or
error reading word contact
error reading word Email
error reading word on
error reading word At
error reading word by
error reading word in
error reading word emailing
error reading word Contact
error reading word at
error reading word •
error reading word at
error reading word is
Found 2195884 word vectors.


In [14]:
########################################
print('Preparing embedding matrix')
nb_words = min(MAX_NB_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing embedding matrix
Null word embeddings: 68661


In [15]:
embedding_matrix.shape

(153189, 300)

In [16]:
embedding_layer = Embedding(nb_words, EMBEDDING_DIM, weights=[embedding_matrix], trainable=False)

In [22]:
comment_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
x = embedding_layer(comment_input)
x = Bidirectional(LSTM(100, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = AveragePooling1D(strides=2)(x)
x = Bidirectional(LSTM(100, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = AveragePooling1D(strides=2)(x)
x = Bidirectional(LSTM(100, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = AveragePooling1D(strides=2)(x)
x = Attention()(x)
x = Dense(200, activation='relu')(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
x = Dense(50, activation='relu')(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
predictions = Dense(6, activation='sigmoid')(x)

In [23]:
model = Model(comment_input, predictions)
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])

In [24]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 300)          45956700  
_________________________________________________________________
bidirectional_4 (Bidirection (None, 100, 200)          320800    
_________________________________________________________________
average_pooling1d_4 (Average (None, 50, 200)           0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 50, 200)           240800    
_________________________________________________________________
average_pooling1d_5 (Average (None, 25, 200)           0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 25, 200)           240800    
__________

In [25]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model_ckpt = ModelCheckpoint(filepath='../models/deep_bstlm_attention.h5', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=5)
callbacks = [early_stopping, model_ckpt, reduce_lr]

In [26]:
model.fit(X_train, y_train, batch_size=256, epochs=50, validation_split=0.1, callbacks=callbacks)

Train on 86265 samples, validate on 9586 samples
Epoch 1/50
86265/86265 [==============================] - 179s 2ms/step - loss: 0.2966 - acc: 0.9096 - val_loss: 0.0729 - val_acc: 0.9794
Epoch 2/50
86265/86265 [==============================] - 174s 2ms/step - loss: 0.0670 - acc: 0.9787 - val_loss: 0.0508 - val_acc: 0.9819
Epoch 3/50
86265/86265 [==============================] - 173s 2ms/step - loss: 0.0545 - acc: 0.9804 - val_loss: 0.0508 - val_acc: 0.9818
Epoch 4/50
86265/86265 [==============================] - 172s 2ms/step - loss: 0.0497 - acc: 0.9815 - val_loss: 0.0509 - val_acc: 0.9809
Epoch 5/50
86265/86265 [==============================] - 173s 2ms/step - loss: 0.0471 - acc: 0.9824 - val_loss: 0.0470 - val_acc: 0.9829
Epoch 6/50
86265/86265 [==============================] - 173s 2ms/step - loss: 0.0453 - acc: 0.9830 - val_loss: 0.0460 - val_acc: 0.9827
Epoch 7/50
86265/86265 [==============================] - 173s 2ms/step - loss: 0.0437 - acc: 0.9835 - val_loss: 0.0458 - v

In [25]:
test = pd.read_csv('../data/test.csv')

In [26]:
test_comments = test['comment_text'].fillna('UNK')
test_sequences = tokenizer.texts_to_sequences(test_comments)
X_test = pad_sequences(test_sequences, maxlen = MAX_SEQUENCE_LENGTH)

In [27]:
model.load_weights('../models/deep_bstlm_attention.h5')

In [28]:
y_pred = model.predict(X_test)

In [30]:
sub = pd.read_csv('../data/sample_submission.csv')

In [37]:
sub.iloc[:, 1:] = y_pred

In [38]:
sub.to_csv('../submissions/deep_bstlm_attention.csv', index=False)

In [40]:
sub.to_csv('bstlm_attention.csv', index=False)

In [39]:
pd.read_csv('../submissions/deep_bstlm_attention.csv').head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,6044863,0.000402,0.000025,0.000044,4.169984e-06,0.000041,0.000004
1,6102620,0.000703,0.000010,0.000268,7.169110e-07,0.000042,0.000008
2,14563293,0.000136,0.000005,0.000077,4.495609e-07,0.000010,0.000003
3,21086297,0.001266,0.000038,0.000113,5.858633e-06,0.000108,0.000009
4,22982444,0.000594,0.000033,0.000063,6.175662e-06,0.000057,0.000006
